# Database Connection with SQLAlchemy

This notebook demonstrates how to connect to a PostgreSQL database using SQLAlchemy.

In [1]:
# Import required libraries
import pandas as pd
from sqlalchemy import create_engine, text
import os
from pprint import pprint

In [2]:
# Database connection parameters
# Update these with your actual database credentials
DB_HOST = '172.17.154.140'
DB_PORT = '5432'
DB_NAME = 'stem_cell_registry'
DB_USER = 'postgres'
DB_PASSWORD = 'password'

# Alternative: Use environment variables for security
# DB_HOST = os.getenv('DB_HOST', 'localhost')
# DB_PORT = os.getenv('DB_PORT', '5432')
# DB_NAME = os.getenv('DB_NAME')
# DB_USER = os.getenv('DB_USER')
# DB_PASSWORD = os.getenv('DB_PASSWORD')

In [3]:
# Create database connection string
connection_string = f'postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}'

# Create SQLAlchemy engine
engine = create_engine(connection_string)

print("Database engine created successfully")

Database engine created successfully


In [4]:
# Test connection
try:
    with engine.connect() as connection:
        result = connection.execute(text("SELECT version();"))
        version = result.fetchone()[0]
        print(f"Connected successfully!")
        print(f"PostgreSQL version: {version}")
except Exception as e:
    print(f"Connection failed: {e}")

Connected successfully!
PostgreSQL version: PostgreSQL 16.10 (Ubuntu 16.10-0ubuntu0.24.04.1) on x86_64-pc-linux-gnu, compiled by gcc (Ubuntu 13.3.0-6ubuntu2~24.04) 13.3.0, 64-bit


In [5]:
# List all tables in the database
try:
    with engine.connect() as connection:
        result = connection.execute(text("""
            SELECT table_name 
            FROM information_schema.tables 
            WHERE table_schema = 'public'
            ORDER BY table_name;
        """))
        tables = [row[0] for row in result.fetchall()]
        print("Available tables:")
        for table in tables:
            print(f"  - {table}")
except Exception as e:
    print(f"Error listing tables: {e}")

Available tables:
  - account_emailaddress
  - account_emailconfirmation
  - auth_group
  - auth_group_permissions
  - auth_permission
  - auth_user
  - auth_user_groups
  - auth_user_user_permissions
  - django_admin_log
  - django_celery_beat_clockedschedule
  - django_celery_beat_crontabschedule
  - django_celery_beat_intervalschedule
  - django_celery_beat_periodictask
  - django_celery_beat_periodictasks
  - django_celery_beat_solarschedule
  - django_celery_results_chordcounter
  - django_celery_results_groupresult
  - django_celery_results_taskresult
  - django_content_type
  - django_migrations
  - django_session
  - ontologies_ontology
  - ontologies_ontologyparentchild
  - ontologies_synonyms
  - socialaccount_socialaccount
  - socialaccount_socialapp
  - socialaccount_socialtoken
  - stakeholders_contact
  - stakeholders_group
  - stakeholders_group_contacts
  - stakeholders_group_institutes
  - stakeholders_institute
  - stakeholders_reguser
  - stem_cell_additionalgenomicc

In [6]:
# Test query 1.
query = "SELECT * from stem_cell_cellline"
df = pd.read_sql(query, engine)
df.head()



,id,hpscreg_name,cell_line_alt_name,cell_type,status,genotype,genotype_locus,frozen,associated_polymorphism,research_use,...,curation_status,biopsy_location_id,culture_medium_id,generator_group_id,genomic_characterisation_id,owner_group_id,registration_requirements_id,screening_contaminates_id,source_id,undifferentiated_characterisation_id
0,302,UOWi004-A,None,hiPSC,CH,PC,,False,,True,...,EXT,None,247.0,43.0,NaN,NaN,392,NaN,NaN,NaN
1,303,MICCNi001-A,None,hiPSC,CH,PC,,False,,True,...,EXT,None,248.0,39.0,89.0,39.0,393,NaN,125.0,86.0
2,28,ESIBIe003-A,None,hiPSC,CH,PC,,False,,True,...,EXT,None,NaN,NaN,NaN,NaN,29,NaN,NaN,NaN
3,23,CIAUi002-B,UBC3-U,hiPSC,CH,PC,,False,,True,...,EXT,None,20.0,4.0,NaN,NaN,230,NaN,NaN,NaN
4,21,CIAUi001-A,AOY7,hiPSC,CH,PC,,False,,True,...,EXT,None,18.0,4.0,NaN,NaN,522,NaN,NaN,NaN


In [7]:
# CellLine object
def create_cellline_object(hpscreg_name: str) -> dict:

    query = f"""SELECT 
    hpscreg_name, cell_line_alt_name, cell_type, frozen
    FROM stem_cell_cellline 
    WHERE hpscreg_name = '{hpscreg_name}'"""

    df = pd.read_sql(query, engine)
    if df.empty:
        raise ValueError(f"No cell line found with name: {hpscreg_name}")
    
    return df.iloc[0].to_dict()



result = create_cellline_object("AIBNi001-A")
print(result)


{'hpscreg_name': 'AIBNi001-A', 'cell_line_alt_name': 'GENIE 1', 'cell_type': 'hiPSC', 'frozen': True}


In [8]:
cell_line_query = f"SELECT * FROM stem_cell_cellline"
cell_line_data = pd.read_sql(query, engine)

cell_line_data.head(20)

cell_line_data.columns

results_0 = []

def wrap_results(df, label_key):
    return df.groupby('hpscreg_name').apply(
        lambda x: {label_key: x.drop('hpscreg_name', axis=1).to_dict('records')}
    ).to_dict()

In [9]:
columns = ['hpscreg_name', 'cell_line_alt_name', 'cell_type', 'frozen']

basic_data = cell_line_data[columns]

result = wrap_results(basic_data, 'basic_data')
pprint(result, indent=2)

results_0.append(result)


{ 'AIBNe001-A': { 'basic_data': [ { 'cell_line_alt_name': 'MEL1(AAVS1::tetON-fastFUCCI)',
                                    'cell_type': 'hiPSC',
                                    'frozen': False}]},
  'AIBNi001-A': { 'basic_data': [ { 'cell_line_alt_name': 'GENIE 1',
                                    'cell_type': 'hiPSC',
                                    'frozen': True}]},
  'AIBNi002-A': { 'basic_data': [ { 'cell_line_alt_name': 'GENIE 38',
                                    'cell_type': 'hiPSC',
                                    'frozen': True}]},
  'AIBNi003-A': { 'basic_data': [ { 'cell_line_alt_name': 'GENIE 68',
                                    'cell_type': 'hiPSC',
                                    'frozen': True}]},
  'AIBNi004-A': { 'basic_data': [ { 'cell_line_alt_name': 'GENIE 69',
                                    'cell_type': 'hiPSC',
                                    'frozen': True}]},
  'AIBNi005-A': { 'basic_data': [ { 'cell_line_alt_name': 'GENIE 

/tmp/ipykernel_2077/3185009912.py:11: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  return df.groupby('hpscreg_name').apply(


In [10]:

group_contacts_query = """ 
SELECT * FROM stakeholders_group 
JOIN stakeholders_group_contacts ON stakeholders_group.id = stakeholders_group_contacts.group_id
JOIN stakeholders_contact ON stakeholders_group_contacts.contact_id = stakeholders_contact.id
"""

group_contacts_data = pd.read_sql(group_contacts_query, engine)
group_contacts_data.head(20)


,id,name,is_research_group,id,group_id,contact_id,id,first_name,last_name,name_initials,e_mail,phone_number
0,62,Wolvetang-AIBN,True,16,62,17,17,Ernst,Wolvetang,J,e.wolvetang@uq.edu.au,None
1,63,Lockhart-MCRI,True,17,63,18,18,Paul,Lockhart,J,paul.lockhart@mcri.edu.au,None
2,64,Lamandé-Shoulders-MCRI,True,18,64,19,19,Shireen,Lamandé,None,shireen.lamande@mcri.edu.au,None
3,64,Lamandé-Shoulders-MCRI,True,19,64,20,20,Matthew,Shoulders,None,mshoulde@mit.edu,None
4,65,Dunwoodie-VCCRI,True,20,65,21,21,Sally,Dunwoodie,None,s.dunwoodie@victorchang.edu.au,None
5,66,Graham-VCCRI,True,21,66,22,22,Robert,Graham,M,b.graham@victorchang.edu.au,None
6,67,North-MCRI,True,22,67,23,23,Kathryn,North,N,Kathryn.north@mcri.edu.au,None
7,68,Bateman-MCRI,True,23,68,25,25,John,Bateman,F,john.bateman@mcri.edu.au,None
8,69,McLenachan-Chen-LEI,True,24,69,26,26,Samuel,McLenachan,None,smclenachan@lei.org.au,None
9,69,McLenachan-Chen-LEI,True,25,69,27,27,Fred,Chen,K,fredchen@lei.org.au,None


In [11]:
from pprint import pprint

# Owner contact details 
df = cell_line_data.merge(group_contacts_data, left_on="owner_group_id", right_on="group_id")
columns = ['hpscreg_name', 'name', 'first_name', 'last_name', 'name_initials', 'e_mail', 'phone_number']
df = df[columns]


df.rename(columns={'name': 'group'}, inplace=True)
result = wrap_results(df, 'contact')


pprint(result, indent=2)

results_0.append(result)



{ 'AIBNi001-A': { 'contact': [ { 'e_mail': ' e.wolvetang@uq.edu.au',
                                 'first_name': 'Ernst',
                                 'group': 'Wolvetang-AIBN',
                                 'last_name': 'Wolvetang',
                                 'name_initials': 'J',
                                 'phone_number': None}]},
  'AIBNi002-A': { 'contact': [ { 'e_mail': ' e.wolvetang@uq.edu.au',
                                 'first_name': 'Ernst',
                                 'group': 'Wolvetang-AIBN',
                                 'last_name': 'Wolvetang',
                                 'name_initials': 'J',
                                 'phone_number': None}]},
  'AIBNi003-A': { 'contact': [ { 'e_mail': ' e.wolvetang@uq.edu.au',
                                 'first_name': 'Ernst',
                                 'group': 'Wolvetang-AIBN',
                                 'last_name': 'Wolvetang',
                                 'name_ini

/tmp/ipykernel_2077/3185009912.py:11: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  return df.groupby('hpscreg_name').apply(


In [12]:
# Generator group object

group = pd.read_sql("SELECT * FROM stakeholders_group", engine)


df2 = cell_line_data.merge(group, left_on="generator_group_id", right_on="id", how='left')
columns = ['hpscreg_name', 'name']
df2 = df2[columns]

df2.rename(columns={'name': 'group'}, inplace=True)
result = wrap_results(df2, 'generator')

pprint(result, indent=2)

results_0.append(result)


/tmp/ipykernel_2077/3185009912.py:11: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  return df.groupby('hpscreg_name').apply(


{ 'AIBNe001-A': { 'generator': [ { 'group': 'Australian Institute for '
                                            'Bioengineering and '
                                            'Nanotechnology'}]},
  'AIBNi001-A': { 'generator': [ { 'group': 'Australian Institute for '
                                            'Bioengineering and '
                                            'Nanotechnology'}]},
  'AIBNi002-A': { 'generator': [ { 'group': 'Australian Institute for '
                                            'Bioengineering and '
                                            'Nanotechnology'}]},
  'AIBNi003-A': { 'generator': [ { 'group': 'Australian Institute for '
                                            'Bioengineering and '
                                            'Nanotechnology'}]},
  'AIBNi004-A': { 'generator': [ { 'group': 'Australian Institute for '
                                            'Bioengineering and '
                                            'Nanot

In [13]:
# Publication table
publication_query = "SELECT * from stem_cell_publication"
publication_data = pd.read_sql(publication_query, engine)
publication_data

junction_query = "SELECT * FROM stem_cell_cellline_publications"
junction = pd.read_sql(junction_query, engine)

mid = junction.merge(publication_data, left_on='publication_id', right_on='id', how='left')

df = cell_line_data.merge(mid, left_on='id', right_on='cellline_id', how='left')

columns = ['hpscreg_name', 'doi', 'journal', 'title', 'first_author', 'last_author', 'year', 'pmid']
df =df[columns]

def clean_float(x):
    """Remove decimal points by converting to string and splitting on '.'"""
    if pd.notna(x):
        return str(x).split('.')[0]
    return x

# Apply clean_float to pmid column
df['pmid'] = df['pmid'].apply(clean_float)
df['year'] = df['year'].apply(clean_float)

result = wrap_results(df, 'publications')
pprint(result, indent=2)

results_0.append(result)

{ 'AIBNe001-A': { 'publications': [ { 'doi': '10.1016/j.scr.2020.102103',
                                      'first_author': 'Ovchinnikov Dmitry A.',
                                      'journal': 'Stem Cell Research',
                                      'last_author': 'Ovchinnikov Dmitry A.',
                                      'pmid': None,
                                      'title': None,
                                      'year': '2020'}]},
  'AIBNi001-A': { 'publications': [ { 'doi': '10.1016/j.scr.2021.102564',
                                      'first_author': 'Hunter ZL',
                                      'journal': 'Stem cell research',
                                      'last_author': 'Vadlamudi L',
                                      'pmid': '34649201',
                                      'title': 'Generation of induced '
                                               'pluripotent stem cell lines '
                                               '

/tmp/ipykernel_2077/3185009912.py:11: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  return df.groupby('hpscreg_name').apply(


In [14]:
# Donor ground truth 

cellline_source = pd.read_sql("SELECT * FROM stem_cell_celllinesource", engine)
donorsource = pd.read_sql("SELECT * FROM stem_cell_donorsource", engine)
donorsource_disease = pd.read_sql("SELECT * FROM stem_cell_donorsource_disease", engine)
disease = pd.read_sql("SELECT * FROM stem_cell_disease", engine)

df2 = donorsource_disease.merge(disease, left_on='disease_id', right_on='id', how='left')
df2 = df2.merge(donorsource, left_on='donorsource_id', right_on='id', how='left')
donor_disease = df2

donor_disease.head(20)

columns = ['donorsource_id', 'age', 'sex', 'description', 'name']
donor_disease = donor_disease[columns]
donor_disease.head(20)

cl_data = cell_line_data[['id', 'hpscreg_name', 'source_id']]

df = cl_data.merge(cellline_source, left_on='source_id', right_on='id', how='left')


df2 = df.merge(donor_disease, left_on='donor_source_id', right_on='donorsource_id', how='left')

df2.head(20)

columns = ['hpscreg_name', 'age', 'sex', 'description', 'name']

df2 = df2[columns]
df2.rename(columns={'name': 'disease_name', 'description': 'disease_description'}, inplace=True)

result = wrap_results(df2, 'donor')
pprint(result, indent=2)

results_0.append(result)


{ 'AIBNe001-A': { 'donor': [ { 'age': nan,
                               'disease_description': nan,
                               'disease_name': nan,
                               'sex': nan}]},
  'AIBNi001-A': { 'donor': [ { 'age': '35_39',
                               'disease_description': 'nan',
                               'disease_name': 'Epilepsy',
                               'sex': 'F'}]},
  'AIBNi002-A': { 'donor': [ { 'age': '45_49',
                               'disease_description': 'nan',
                               'disease_name': 'Epilepsy',
                               'sex': 'F'}]},
  'AIBNi003-A': { 'donor': [ { 'age': '15_19',
                               'disease_description': 'nan',
                               'disease_name': 'Epilepsy',
                               'sex': 'F'}]},
  'AIBNi004-A': { 'donor': [ { 'age': '25_29',
                               'disease_description': 'nan',
                               'disease_name': 'Epile

/tmp/ipykernel_2077/3185009912.py:11: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  return df.groupby('hpscreg_name').apply(


In [15]:
genomic_alteration = pd.read_sql("SELECT * FROM stem_cell_genomicalteration", engine)
loci = pd.read_sql("SELECT * FROM stem_cell_loci", engine)
df0 = genomic_alteration.merge(loci, left_on='loci_id', right_on='id', how='left')

df0 = df0[['id_x', 'name', 'description', 'mutation_type', 'cytoband', 'delivery_method', 'genotype']]
df0.rename(columns={'id_x': 'id', 'name': 'loci_name'}, inplace=True)

cellline_genomic_modifications = pd.read_sql("SELECT * FROM stem_cell_cellline_genomic_modifications", engine)


df = cellline_genomic_modifications.merge(df0, left_on='genomicalteration_id', right_on='id', how='left')

cl_data = cell_line_data[['id', 'hpscreg_name']]


df2 = cl_data.merge(df, left_on='id', right_on='cellline_id', how='left')

df2.head(20)
columns = ['hpscreg_name', 'mutation_type', 'cytoband', 'delivery_method', 'description', 'genotype', 'loci_name']
df2 = df2[columns]

df2.head(20)

result = wrap_results(df2, 'genomic_modifications')
pprint(result, indent=2)


results_0.append(result)

{ 'AIBNe001-A': { 'genomic_modifications': [ { 'cytoband': nan,
                                               'delivery_method': nan,
                                               'description': nan,
                                               'genotype': nan,
                                               'loci_name': nan,
                                               'mutation_type': nan}]},
  'AIBNi001-A': { 'genomic_modifications': [ { 'cytoband': nan,
                                               'delivery_method': nan,
                                               'description': nan,
                                               'genotype': nan,
                                               'loci_name': nan,
                                               'mutation_type': nan}]},
  'AIBNi002-A': { 'genomic_modifications': [ { 'cytoband': nan,
                                               'delivery_method': nan,
                                               'description

/tmp/ipykernel_2077/3185009912.py:11: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  return df.groupby('hpscreg_name').apply(


In [16]:
# Differentiation results

characterisationprotocolresult = pd.read_sql("SELECT * FROM stem_cell_characterisationprotocolresult", engine)
characterisationmethod = pd.read_sql("SELECT * FROM stem_cell_characterisationmethod", engine)

cl_data = cell_line_data[['id', 'hpscreg_name']]

df1 = cl_data.merge(characterisationprotocolresult, left_on='id', right_on='cell_line_id', how='left')
df2 = df1.merge(characterisationmethod, left_on='method_id', right_on='id', how='left')
df2.head(20)

columns = ['hpscreg_name', 'cell_type', 'show_potency', 'marker_list', 'name', 'description']
df2 = df2[columns]
df2.rename(columns={'name': 'method_used'}, inplace=True)

result = wrap_results(df2, 'differentiation_results')
pprint(result, indent=2)


results_0.append(result)

{ 'AIBNe001-A': { 'differentiation_results': [ { 'cell_type': nan,
                                                 'description': nan,
                                                 'marker_list': nan,
                                                 'method_used': nan,
                                                 'show_potency': nan}]},
  'AIBNi001-A': { 'differentiation_results': [ { 'cell_type': 'EN',
                                                 'description': '',
                                                 'marker_list': 'FOXA2; SOX17',
                                                 'method_used': 'RT-qPCR',
                                                 'show_potency': True},
                                               { 'cell_type': 'ME',
                                                 'description': '',
                                                 'marker_list': 'HAND1; RUNX1',
                                                 'method_used': 'RT-qPCR',

/tmp/ipykernel_2077/3185009912.py:11: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  return df.groupby('hpscreg_name').apply(


In [17]:
undifferentiationcharactrisation = pd.read_sql("SELECT * FROM stem_cell_undifferentiatedcharacterisation", engine)

cl_data = cell_line_data[['id', 'hpscreg_name', 'undifferentiated_characterisation_id']]

df = cl_data.merge(undifferentiationcharactrisation, left_on='undifferentiated_characterisation_id', right_on='id', how='left')

columns = ['hpscreg_name', 'epi_pluri_score', 'pluri_test_score', 'pluri_novelty_score']

df2 = df[columns]

result = wrap_results(df2, 'undifferentiated_characterisation')
pprint(result, indent=2)


results_0.append(result)



{ 'AIBNe001-A': { 'undifferentiated_characterisation': [ { 'epi_pluri_score': nan,
                                                           'pluri_novelty_score': nan,
                                                           'pluri_test_score': nan}]},
  'AIBNi001-A': { 'undifferentiated_characterisation': [ { 'epi_pluri_score': None,
                                                           'pluri_novelty_score': nan,
                                                           'pluri_test_score': nan}]},
  'AIBNi002-A': { 'undifferentiated_characterisation': [ { 'epi_pluri_score': None,
                                                           'pluri_novelty_score': nan,
                                                           'pluri_test_score': nan}]},
  'AIBNi003-A': { 'undifferentiated_characterisation': [ { 'epi_pluri_score': None,
                                                           'pluri_novelty_score': nan,
                                                        

/tmp/ipykernel_2077/3185009912.py:11: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  return df.groupby('hpscreg_name').apply(


In [18]:
# Genomic characterisation
genomiccharacterisation = pd.read_sql("SELECT * FROM stem_cell_genomiccharacterisation", engine)

cl_data = cell_line_data[['id', 'hpscreg_name', 'genomic_characterisation_id']]

df = cl_data.merge(genomiccharacterisation, left_on='genomic_characterisation_id', right_on='id', how='left')

columns = ['hpscreg_name', 'passage_number', 'karyotype', 'karyotype_method', 'summary']

df2 = df[columns]

result = wrap_results(df2, 'genomic_characterisation')
pprint(result, indent=2)


results_0.append(result)


{ 'AIBNe001-A': { 'genomic_characterisation': [ { 'karyotype': nan,
                                                  'karyotype_method': nan,
                                                  'passage_number': nan,
                                                  'summary': nan}]},
  'AIBNi001-A': { 'genomic_characterisation': [ { 'karyotype': '46,XX',
                                                  'karyotype_method': 'GB',
                                                  'passage_number': '9',
                                                  'summary': 'Karyotyping '
                                                             'occurred at a '
                                                             'resolution of '
                                                             '300bphs. Fifteen '
                                                             'metaphase '
                                                             'spreads were '
                              

/tmp/ipykernel_2077/3185009912.py:11: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  return df.groupby('hpscreg_name').apply(


In [19]:
# Induced derivation information
derivationinducedpluripotent = pd.read_sql("SELECT * FROM stem_cell_celllinederivationinducedpluripotent", engine)
nonintegratedvector = pd.read_sql("SELECT * FROM stem_cell_nonintegratedvector", engine)

cl_data = cell_line_data[['id', 'hpscreg_name']]

df1 = derivationinducedpluripotent.merge(nonintegratedvector, left_on='non_integrated_vector_id', right_on='id', how='left')

df2 = cl_data.merge(df1, left_on='id', right_on='cell_line_id', how='left')

columns = ['hpscreg_name', 'i_source_cell_type_id', 'i_source_cell_origin_id', 'derivation_year', 'non_int_vector', 'non_int_vector_name']



df2 = df2[columns]


df2['derivation_year'] = df2['derivation_year'].astype(str)


result = wrap_results(df2, 'induced_derivation')
pprint(result, indent=2)

results_0.append(result)

{ 'AIBNe001-A': { 'induced_derivation': [ { 'derivation_year': 'None',
                                            'i_source_cell_origin_id': None,
                                            'i_source_cell_type_id': None,
                                            'non_int_vector': nan,
                                            'non_int_vector_name': nan}]},
  'AIBNi001-A': { 'induced_derivation': [ { 'derivation_year': '2020-01-01',
                                            'i_source_cell_origin_id': 'UBERON:0000178',
                                            'i_source_cell_type_id': 'CL:2000001',
                                            'non_int_vector': 'SV',
                                            'non_int_vector_name': '[kit] '
                                                                   'CytoTune-iPS '
                                                                   '2.0 Sendai '
                                                                   'reprogramm

/tmp/ipykernel_2077/3185009912.py:11: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  return df.groupby('hpscreg_name').apply(


In [20]:
# Embryonic stem cell information
celllinederivationembryonic = pd.read_sql("SELECT * FROM stem_cell_celllinederivationembryonic", engine)

cl_data = cell_line_data[['id', 'hpscreg_name']]

df1 = cl_data.merge(celllinederivationembryonic, left_on='id', right_on='cell_line_id', how='left')

columns = ['hpscreg_name', 'embryo_stage', 'zp_removal_technique', 'trophectoderm_morphology', 'icm_morphology', 'e_preimplant_genetic_diagnosis']

df2 = df1[columns]

result = wrap_results(df2, 'embryonic_derivation')
pprint(result, indent=2)


results_0.append(result)

{ 'AIBNe001-A': { 'embryonic_derivation': [ { 'e_preimplant_genetic_diagnosis': nan,
                                              'embryo_stage': nan,
                                              'icm_morphology': nan,
                                              'trophectoderm_morphology': nan,
                                              'zp_removal_technique': nan}]},
  'AIBNi001-A': { 'embryonic_derivation': [ { 'e_preimplant_genetic_diagnosis': nan,
                                              'embryo_stage': nan,
                                              'icm_morphology': nan,
                                              'trophectoderm_morphology': nan,
                                              'zp_removal_technique': nan}]},
  'AIBNi002-A': { 'embryonic_derivation': [ { 'e_preimplant_genetic_diagnosis': nan,
                                              'embryo_stage': nan,
                                              'icm_morphology': nan,
                       

/tmp/ipykernel_2077/3185009912.py:11: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  return df.groupby('hpscreg_name').apply(


In [21]:
ethics = pd.read_sql("SELECT * FROM stem_cell_ethics", engine)
registrationrequirements = pd.read_sql("SELECT * FROM stem_cell_registrationrequirements", engine)



cl_data = cell_line_data[['id', 'hpscreg_name', 'registration_requirements_id']]
df1 = cl_data.merge(registrationrequirements, left_on='registration_requirements_id', right_on='id', how='left')
df2 = df1.merge(ethics, left_on='ethics_id', right_on='id', how='left')
columns = ['hpscreg_name', 'ethics_number', 'approval_date', 'institutional_HREC']


df2 = df2[columns]

df2['approval_date'] = df2['approval_date'].astype(str)


result = wrap_results(df2, 'ethics')
pprint(result, indent=2)


results_0.append(result)


{ 'AIBNe001-A': { 'ethics': [ { 'approval_date': 'None',
                                'ethics_number': 'Holding Entry-AIBNe001-A',
                                'institutional_HREC': 'Holding Entry'}]},
  'AIBNi001-A': { 'ethics': [ { 'approval_date': '2019-01-01',
                                'ethics_number': 'QRBW/54086',
                                'institutional_HREC': 'The University of '
                                                      'Queensland Human Ethics '
                                                      'Research Office'}]},
  'AIBNi002-A': { 'ethics': [ { 'approval_date': '2019-01-01',
                                'ethics_number': 'QRBW/54086',
                                'institutional_HREC': 'The University of '
                                                      'Queensland Human Ethics '
                                                      'Research Office'}]},
  'AIBNi003-A': { 'ethics': [ { 'approval_date': '2019-01-01',
             

/tmp/ipykernel_2077/3185009912.py:11: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  return df.groupby('hpscreg_name').apply(


In [22]:
culturemedium = pd.read_sql("SELECT * FROM stem_cell_culturemedium", engine)

cl_data = cell_line_data[['id', 'hpscreg_name', 'culture_medium_id']]

df = cl_data.merge(culturemedium, left_on='culture_medium_id', right_on='id', how='left')

columns = ['hpscreg_name', 'co2_concentration', 'o2_concentration', 'passage_method']

df2 = df[columns]

result = wrap_results(df2, 'culture_medium')
pprint(result, indent=2)

results_0.append(result)

{ 'AIBNe001-A': { 'culture_medium': [ { 'co2_concentration': nan,
                                        'o2_concentration': nan,
                                        'passage_method': 'NS'}]},
  'AIBNi001-A': { 'culture_medium': [ { 'co2_concentration': 0.05,
                                        'o2_concentration': nan,
                                        'passage_method': 'EF'}]},
  'AIBNi002-A': { 'culture_medium': [ { 'co2_concentration': 0.05,
                                        'o2_concentration': nan,
                                        'passage_method': 'EF'}]},
  'AIBNi003-A': { 'culture_medium': [ { 'co2_concentration': 0.05,
                                        'o2_concentration': nan,
                                        'passage_method': 'EF'}]},
  'AIBNi004-A': { 'culture_medium': [ { 'co2_concentration': 0.05,
                                        'o2_concentration': nan,
                                        'passage_method': 'EF'}]},
  'AIB

/tmp/ipykernel_2077/3185009912.py:11: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  return df.groupby('hpscreg_name').apply(


In [23]:
results_0[0]

{'AIBNe001-A': {'basic_data': [{'cell_line_alt_name': 'MEL1(AAVS1::tetON-fastFUCCI)',
    'cell_type': 'hiPSC',
    'frozen': False}]},
 'AIBNi001-A': {'basic_data': [{'cell_line_alt_name': 'GENIE 1',
    'cell_type': 'hiPSC',
    'frozen': True}]},
 'AIBNi002-A': {'basic_data': [{'cell_line_alt_name': 'GENIE 38',
    'cell_type': 'hiPSC',
    'frozen': True}]},
 'AIBNi003-A': {'basic_data': [{'cell_line_alt_name': 'GENIE 68',
    'cell_type': 'hiPSC',
    'frozen': True}]},
 'AIBNi004-A': {'basic_data': [{'cell_line_alt_name': 'GENIE 69',
    'cell_type': 'hiPSC',
    'frozen': True}]},
 'AIBNi005-A': {'basic_data': [{'cell_line_alt_name': 'GENIE 70',
    'cell_type': 'hiPSC',
    'frozen': True}]},
 'AIBNi006-A': {'basic_data': [{'cell_line_alt_name': 'GENIE 74',
    'cell_type': 'hiPSC',
    'frozen': True}]},
 'AIBNi007-A': {'basic_data': [{'cell_line_alt_name': 'GENIE 11',
    'cell_type': 'hiPSC',
    'frozen': True}]},
 'AIBNi008-A': {'basic_data': [{'cell_line_alt_name': 'GENIE

In [24]:
hpscreg_names = cell_line_data['hpscreg_name'].unique()
hpscreg_names = sorted(hpscreg_names)


In [25]:

from collections import defaultdict
gt_data = defaultdict(dict)

for hpscreg_name in hpscreg_names[:]:

    #print("Processing cell line: ", hpscreg_name)

    for metadata_package in results_0[:]:

        #print(metadata_package)
        if hpscreg_name not in metadata_package:
            continue
        
        package = metadata_package[hpscreg_name]
        metadata_name, metadata_list = list(package.items())[0]
        gt_data[hpscreg_name][metadata_name] = metadata_list




        
  

In [26]:
pprint(gt_data['AIBNi001-A'])

{'basic_data': [{'cell_line_alt_name': 'GENIE 1',
                 'cell_type': 'hiPSC',
                 'frozen': True}],
 'contact': [{'e_mail': ' e.wolvetang@uq.edu.au',
              'first_name': 'Ernst',
              'group': 'Wolvetang-AIBN',
              'last_name': 'Wolvetang',
              'name_initials': 'J',
              'phone_number': None}],
 'culture_medium': [{'co2_concentration': 0.05,
                     'o2_concentration': nan,
                     'passage_method': 'EF'}],
 'differentiation_results': [{'cell_type': 'EN',
                              'description': '',
                              'marker_list': 'FOXA2; SOX17',
                              'method_used': 'RT-qPCR',
                              'show_potency': True},
                             {'cell_type': 'ME',
                              'description': '',
                              'marker_list': 'HAND1; RUNX1',
                              'method_used': 'RT-qPCR',
          

In [27]:
def convert_to_strings(obj):
    """Convert all values in nested dict/list structure to strings.

    Rules:
    - None -> "Missing"
    - True -> "True"
    - False -> "False"
    - nan -> "Missing"
    - Everything else -> str(value)
    """
    import pandas as pd

    if isinstance(obj, dict):
        return {key: convert_to_strings(value) for key, value in obj.items()}
    elif isinstance(obj, list):
        return [convert_to_strings(item) for item in obj]
    elif obj is None:
        return "Missing"
    elif isinstance(obj, bool):
        return str(obj)
    elif pd.isna(obj):
        return "Missing"
    else:
        return str(obj)

for hpscreg_name in hpscreg_names:
    # Convert all values to strings and use none type "Missing"
    string_data = convert_to_strings(gt_data[hpscreg_name])
    gt_data[hpscreg_name] = string_data
 

In [28]:
# Cell type and tissue type OLS lookups 
import requests

def get_tissue_term(short_form):
    base_url = 'https://www.ebi.ac.uk/ols4'
    params = {
        'short_form': short_form,
    }
    response = requests.get(f'{base_url}/api/ontologies/uberon/terms', params=params).json()
    return response['_embedded']['terms'][0]['label']


def get_cell_type_term(short_form):
    base_url = 'https://www.ebi.ac.uk/ols4'
    params = {
        'short_form': short_form,
    }
    response = requests.get(f'{base_url}/api/ontologies/cl/terms', params=params).json()
    return response['_embedded']['terms'][0]['label']



In [29]:
# Cell type and tissue type OLS lookups 
failed_retrievals = []
for cell_line, metadata in gt_data.items():
    print('Processing cell line: ', cell_line)
    source_cell_type = metadata['induced_derivation'][0]['i_source_cell_type_id']
    source_tissue_type = metadata['induced_derivation'][0]['i_source_cell_origin_id']
    
    print("Source cell type: ", source_cell_type)
    print("Source tissue type: ", source_tissue_type)

    if cell_type_term != "Missing":
        try:
            cell_type_term = get_cell_type_term(source_cell_type)
        except Exception as e:
            failed_retrievals.append(cell_line)
            continue
    if tissue_type_term != "Missing":
        try:
            tissue_type_term = get_tissue_term(source_tissue_type)
        except Exception as e:
            failed_retrievals.append(cell_line)
    

    print('Cell type term: ', cell_type_term)
    print('Tissue type term: ', tissue_type_term)
    print("Finished processing cell line: ", cell_line)  

    metadata['induced_derivation'][0]['i_source_cell_type_term'] = cell_type_term
    metadata['induced_derivation'][0]['i_source_cell_origin_term'] = tissue_type_term


Processing cell line:  AIBNe001-A
Source cell type:  Missing
Source tissue type:  Missing


NameError: name 'cell_type_term' is not defined

In [ ]:
gt_data['AIBNi001-A']

{'basic_data': [{'cell_line_alt_name': 'GENIE 1',
   'cell_type': 'hiPSC',
   'frozen': 'True'}],
 'contact': [{'group': 'Wolvetang-AIBN',
   'first_name': 'Ernst',
   'last_name': 'Wolvetang',
   'name_initials': 'J',
   'e_mail': ' e.wolvetang@uq.edu.au',
   'phone_number': 'Missing'}],
 'generator': [{'group': 'Australian Institute for Bioengineering and Nanotechnology'}],
 'publications': [{'doi': '10.1016/j.scr.2021.102564',
   'journal': 'Stem cell research',
   'title': 'Generation of induced pluripotent stem cell lines from peripheral blood mononuclear cells of three drug resistant and three drug responsive epilepsy patients',
   'first_author': 'Hunter ZL',
   'last_author': 'Vadlamudi L',
   'year': '2021',
   'pmid': '34649201'},
  {'doi': 'https://doi.org/10.1016/j.scr.2021.102564',
   'journal': 'Stem Cell Research',
   'title': 'Generation of induced pluripotent stem cell lines from peripheral blood mononuclear cells of three drug resistant and three drug responsive epile

In [ ]:
hpscreg_names = list(gt_data.keys())
hpscreg_names = sorted(hpscreg_names)

for hpscreg_name in hpscreg_names:
    string_data = convert_to_strings(gt_data[hpscreg_name])
    gt_data[hpscreg_name] = string_data



In [526]:
hpscreg_names = list(gt_data.keys())
hpscreg_names = sorted(hpscreg_names)

import json
for hpscreg_name in hpscreg_names:
    with open(f"ground_truth/{hpscreg_name}_gt.json", "w") as f:
        string_data = convert_to_strings(gt_data[hpscreg_name])
        json.dump(string_data, f, indent=2)

Processing 2

In [527]:
# Close the connection when done
engine.dispose()
print("Database connection closed")

Database connection closed
